* Choose the correct _kernel_
* The kernel is the underlying environment for the Jupyter Notebook.  
* The right kernel will be built to interact with the GPUs.  It does so in a way that is (somewhat) transparent to us.


In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Find out specificaitions of the GPU(s)
if torch.cuda.is_available():
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/(1024)**3)
    
    device_type = "cuda:" + str(torch.cuda.current_device())
else:
    device_type = "cpu"
    # device_type = "mps"    # for macos
    
device = torch.device(device_type)
print("Device:  {}".format(device))

#### Read the Data

In [ ]:
## We create dataloader with dataset from torchvision, 
## and we dont have to download it seperately, all automatically done
# DataLoader: passes samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting, 
# and use Python’s multiprocessing to speed up data retrieval.
# Define batch size, batch size is how much data you feed for training in one iteration
batch_size_train = 64 # 
batch_size_test = 1024 #

# define how image transformed
image_transform = torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))     # mean and std for each channel
                             ])
#image datasets
train_dataset = torchvision.datasets.MNIST('dataset/', 
                                           train=True, 
                                           download=True,
                                           transform=image_transform)
test_dataset = torchvision.datasets.MNIST('dataset/', 
                                          train=False, 
                                          download=True,
                                          transform=image_transform)
#data loaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size_train, 
                                           shuffle=True,
                                           #num_workers=int(os.environ["SLURM_CPUS_PER_TASK"]
                                           )
                                          
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size_test, 
                                          shuffle=True,
                                          #num_workers=int(os.environ["SLURM_CPUS_PER_TASK"])
                                         )

In [ ]:
# We can check the dataloader
_, (example_datas, labels) = next(enumerate(test_loader))    # enumerate outputs a list, next picks the next item of the list
sample = example_datas[0][0]     # first index is the size of the batch
# show the data
plt.imshow(sample, cmap='gray', interpolation='none')
print("Label: "+ str(labels[0]))

#### Design the Model

In [ ]:
## Define the model class
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()                # Allows access to methods of the base class
        #input channel 1, output channel 32
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        #input channel 32, output channel 64
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        #dropout layer
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        #fully connected layer
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x,dim=1)
        return output

#### Define Functions for Learning Process

In [ ]:
## create model and optimizer
learning_rate = 0.01
momentum = 0.5
model = CNN().to(device)       # both model and data should be on the same device, otherwise runtime error
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)


In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
%%time
num_epoch = 5
for epoch in range(1, num_epoch + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)


In [ ]:
# pip install --user torchsummary
from torchsummary import summary
summary(model, (1, 28, 28))
# print(model)

#### Performance Tuning
It is important to optimize your script for the single-GPU case before moving to multi-GPU training. This is because as you request more resources, your queue time increases. 
Do not use multiple GPUs if your GPU efficiency is low.


In [ ]:
# Profiling using line_profiler can also be performed using magic words 
from line_profiler import LineProfiler
lp = LineProfiler()
lp.add_function(train)
lp.run('train(model, device, train_loader, optimizer, epoch=1)')
lp.print_stats()